<h1><center>Project 2 LOS</h1>
<h2><center>Pre-Processing</h2>
<h4><center>TCSS 555<br>
Spring 2018<br>
by: Thuan Lam, Tood Robbins, Inno Estrera</h4></center>


<h2>Libraries</h2>

In [1]:
# Load libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from dateutil.parser import parse
from datetime import datetime

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Load Model Datasets
<h1>Data set descriptions</h1>
<h3>https://mimic.physionet.org/mimictables/admissions/</h3>

In [2]:
import os 
cwd = os.getcwd()
print('Current folder is {}'.format(cwd))

Current folder is /Users/innoestrera/repositories/Project-2-Length-Of-Stay/phase 1


### Read files

In [3]:
dataframes = []
if os.name == "posix":
    unreadable = set([
        "LABEVENTS.csv",
        "INPUTEVENTS_CV.csv",
        "DATETIMEEVENTS.csv",
        "PRESCRIPTIONS.csv",
        "NOTEEVENTS.csv",
        "OUTPUTEVENTS.csv",
        "CHARTEVENTS.csv",
        "D_LABITEMS.csv",
        "CPTEVENTS.csv"
    ])
    USER_DIR = '/Users/innoestrera/Desktop/mimic3';
else:
    unreadable = set([])
    USER_DIR = 'C:\mimic3';

for dirpath, dirnames, filenames in os.walk(USER_DIR):
    dataSets = filenames

for fileName in dataSets:
    if fileName != ".DS_Store" and fileName not in unreadable:
        if os.name == "posix":
            csv = USER_DIR + "/{}".format(fileName);
        else:
            csv = USER_DIR + "\{}".format(fileName);
        print("Reading {}".format(csv))
        data = {}
        data['name'] = fileName
        data['table'] = pd.read_csv(csv)
        dataframes.append(data)
print("finished loading all data")

Reading /Users/innoestrera/Desktop/mimic3/PROCEDUREEVENTS_MV.csv
Reading /Users/innoestrera/Desktop/mimic3/CALLOUT.csv
Reading /Users/innoestrera/Desktop/mimic3/D_CPT.csv
Reading /Users/innoestrera/Desktop/mimic3/D_ITEMS.csv
Reading /Users/innoestrera/Desktop/mimic3/CAREGIVERS.csv
Reading /Users/innoestrera/Desktop/mimic3/MICROBIOLOGYEVENTS.csv
Reading /Users/innoestrera/Desktop/mimic3/ADMISSIONS.csv
Reading /Users/innoestrera/Desktop/mimic3/PROCEDURES_ICD.csv
Reading /Users/innoestrera/Desktop/mimic3/TRANSFERS.csv
Reading /Users/innoestrera/Desktop/mimic3/DIAGNOSES_ICD.csv
Reading /Users/innoestrera/Desktop/mimic3/SERVICES.csv
Reading /Users/innoestrera/Desktop/mimic3/DRGCODES.csv
Reading /Users/innoestrera/Desktop/mimic3/PATIENTS.csv
Reading /Users/innoestrera/Desktop/mimic3/D_ICD_DIAGNOSES.csv
Reading /Users/innoestrera/Desktop/mimic3/ICUSTAYS.csv
Reading /Users/innoestrera/Desktop/mimic3/INPUTEVENTS_MV.csv
Reading /Users/innoestrera/Desktop/mimic3/D_ICD_PROCEDURES.csv
finished load

### Shuffling data across all data frames

In [4]:
print("========shuffling all data frames========")
for dataframe in dataframes:
    print("shuffling {}...".format(dataframe['name']))
    shuffle(dataframe['table'])
print("========finished shuffling all data frames========")

========shuffling all data frames========
shuffling PROCEDUREEVENTS_MV.csv...
shuffling CALLOUT.csv...
shuffling D_CPT.csv...
shuffling D_ITEMS.csv...
shuffling CAREGIVERS.csv...
shuffling MICROBIOLOGYEVENTS.csv...
shuffling ADMISSIONS.csv...
shuffling PROCEDURES_ICD.csv...
shuffling TRANSFERS.csv...
shuffling DIAGNOSES_ICD.csv...
shuffling SERVICES.csv...
shuffling DRGCODES.csv...
shuffling PATIENTS.csv...
shuffling D_ICD_DIAGNOSES.csv...
shuffling ICUSTAYS.csv...
shuffling INPUTEVENTS_MV.csv...
shuffling D_ICD_PROCEDURES.csv...
========finished shuffling all data frames========


### Show count for null values

In [5]:
for dataframe in dataframes:
    name = dataframe['name']
    frame = dataframe['table']
    print("{} nulls below".format(name))
    print(frame.isnull().sum())
    print("Total rows: {}".format(len(frame.index)))
    print("========================")

PROCEDUREEVENTS_MV.csv nulls below
ROW_ID                             0
SUBJECT_ID                         0
HADM_ID                            0
ICUSTAY_ID                        88
STARTTIME                          0
ENDTIME                            0
ITEMID                             0
VALUE                              0
VALUEUOM                           0
LOCATION                      205454
LOCATIONCATEGORY              205454
STORETIME                          0
CGID                               0
ORDERID                            0
LINKORDERID                        0
ORDERCATEGORYNAME                  0
SECONDARYORDERCATEGORYNAME    258066
ORDERCATEGORYDESCRIPTION           0
ISOPENBAG                          0
CONTINUEINNEXTDEPT                 0
CANCELREASON                       0
STATUSDESCRIPTION                  0
COMMENTS_EDITEDBY             255973
COMMENTS_CANCELEDBY           252377
COMMENTS_DATE                 250284
dtype: int64
Total rows: 258066
CALLOUT.

### Lower case all columns

In [21]:
print("========lowercasing all columns across all frames========")
for dataframe in dataframes:
    dataframe['table'].columns = map(str.lower, dataframe['table'].columns)
print("========finished lowercasing all columns========")


unneededColumns = [
    "row_id",
    "location",
    "locationcategory",
    "secondaryordercategoryname",
    "comments_editedby",
    "comments_canceledby"
    "edouttime",
    "hospital_expire_flag",
    "has_chartevents_data",
    "dod",
    "dod_hosp",
    "dod_ssn",
    "expire_flag",
    "deathtime"
];

print("========Dropping {} columns across all frames========".format(unneededColumns))
for dataframe in dataframes:
    for column in dataframe['table'].columns.values:
        if column in unneededColumns:
            dataframe['table'] = dataframe['table'].drop(column, axis=1)
print("========finished dropping unneeded columns {}========".format(unneededColumns))

========lowercasing all columns across all frames========
========finished lowercasing all columns========
========Dropping ['row_id', 'location', 'locationcategory', 'secondaryordercategoryname', 'comments_editedby', 'comments_canceledbyedouttime', 'hospital_expire_flag', 'has_chartevents_data', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'deathtime'] columns across all frames========
========finished dropping unneeded columns ['row_id', 'location', 'locationcategory', 'secondaryordercategoryname', 'comments_editedby', 'comments_canceledbyedouttime', 'hospital_expire_flag', 'has_chartevents_data', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'deathtime']========


### Dropping columns with null values

In [22]:
requiredColumns=['subject_id', 'hadm_id', 'icustay_id']

print("==========Dropping rows with null values for required keys {} ==========".format(requiredColumns))
for obj in dataframes:
    df = obj['table']
    for column in df.columns.values:
        if column in requiredColumns:
            df = df[df[column].notnull()];
print("==========All rows with null values have been dropped {}==========".format(requiredColumns))

==========Dropping rows with null values for required keys ['subject_id', 'hadm_id', 'icustay_id'] ==========
==========All rows with null values have been dropped ['subject_id', 'hadm_id', 'icustay_id']==========


### Merging Tables

In [23]:
primaryTables = [
    "ADMISSIONS.csv",
    "PATIENTS.csv"
];

for obj in dataframes:
    name = obj['name']
    df = obj['table']
    if name == "ICUSTAYS.csv":
        masterTable = df.groupby(['subject_id'], as_index=False)['los'].sum()
        break;

for obj in dataframes:
    name = obj['name']
    df = obj['table']
    if name in primaryTables:
        masterTable = pd.merge(masterTable, df, on='subject_id', how='outer');

masterTable = masterTable[masterTable['los'].notnull()]
print(masterTable.isnull().sum())
print(masterTable.shape)

subject_id                0
los                       0
hadm_id                   0
admittime                 0
dischtime                 0
admission_type            0
admission_location        0
discharge_location        0
insurance                 0
language              25304
religion                457
marital_status        10121
ethnicity                 0
edregtime             28072
edouttime             28072
diagnosis                24
gender                    0
dob                       0
dtype: int64
(58931, 18)


In [ ]:
# for dataframe in dataframes:
#     print("========{}========".format(dataframe['name']))
#     dataframe['table'].info()
#     print("========{}========".format(dataframe['name']))

In [16]:
# print("========Creating Master Table========")
# masterTable = pd.DataFrame(columns=['subject_id', 'LOS'])

# tablesWithSubjectId = set([
#     "PROCEDUREEVENTS_MV.csv",
#     "CALLOUT.csv",
#     "MICROBIOLOGYEVENTS.csv",
#     "ADMISSIONS.csv",
#     "PROCEDURES_ICD.csv",
#     "TRANSFERS.csv",
#     "DIAGNOSES_ICD.csv",
#     "SERVICES.csv",
#     "DRGCODES.csv",
#     "PATIENTS.csv",
#     "ICUSTAYS.csv",
#     "INPUTEVENTS_MV.csv"
# ]);

# for dataframe in dataframes:
#     if dataframe['name'] in tablesWithSubjectId:
#         print("merging {} to masterTable on subject_id...".format(dataframe['name']))
#         masterTable = pd.merge(masterTable, dataframe["table"], on='subject_id', how='outer')
# print("========Finished Merging Tables to MasterTable on SUBJECT_ID========")
# print(masterTable.shape)

========Creating Master Table========
merging PROCEDUREEVENTS_MV.csv to masterTable on subject_id...
merging CALLOUT.csv to masterTable on subject_id...
merging MICROBIOLOGYEVENTS.csv to masterTable on subject_id...
merging ADMISSIONS.csv to masterTable on subject_id...


KeyboardInterrupt: 

In [ ]:

# join 
#   ICUSTAYS.csv, 
#   INPUTEVENTS_MV.csv, 
#   PROCEDUREEVENTS_MV.csv, 
#   TRANSFERS.csv
# on ICUSTAY_ID


# join
#   PROCEDUREEVENTS_MV.csv,
#   CALLOUT.csv,
#   MICROBIOLOGYEVENTS.csv,
#   ADMISSIONS.csv,
#   PROCEDURES_ICD.csv,
#   TRANSFERS.csv,
#   DIAGNOSES_ICD.csv,
#   SERVICES.csv,
#   DRGCODES.csv,
#   PATIENTS.csv,
#   ICUSTAYS.csv,
#   INPUTEVENTS_MV.csv
# on 
#   SUBJECT_ID


# join
#   PROCEDUREEVENTS_MV.csv,
#   CALLOUT.csv,
#   MICROBIOLOGYEVENTS.csv,
#   ADMISSIONS.csv,
#   PROCEDURES_ICD.csv,
#   TRANSFERS.csv,
#   DIAGNOSES_ICD.csv,
#   SERVICES.csv,
#   DRGCODES.csv,
#   ICUSTAYS.csv,
#   INPUTEVENTS_MV.csv
# on
#   HADM_ID

## Examine datasets

In [16]:
masterTable.sample(n=20)

,subject_id,los,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
23160,19007,0.2492,104150,2108-08-17 21:39:00,2108-08-21 11:26:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,NEWBORN,0,1,M,2108-08-17 00:00:00,NaN,NaN,NaN,0
37399,30743,17.4839,170722,2135-10-02 02:50:00,2135-10-19 14:45:00,NaN,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,...,NaN,NEWBORN,0,1,F,2135-10-02 00:00:00,NaN,NaN,NaN,0
36095,29631,20.0607,141170,2180-12-04 21:33:00,2180-12-13 14:39:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2180-12-04 22:22:00,UNRESPONSIVE,0,1,M,2101-06-09 00:00:00,2182-12-05 00:00:00,NaN,2182-12-05 00:00:00,1
24470,20068,3.8791,190986,2145-04-30 20:24:00,2145-05-06 03:30:00,2145-05-06 03:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,2145-04-30 21:15:00,CEREBROVASCULAR ACCIDENT,1,1,F,1845-04-30 00:00:00,2145-05-06 00:00:00,2145-05-06 00:00:00,2145-05-06 00:00:00,1
16027,13072,3.3095,184787,2184-08-18 08:00:00,2184-08-21 10:41:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,TUBERCULUM MENINGIOMA/SDA,0,1,F,2146-10-09 00:00:00,NaN,NaN,NaN,0
21899,17944,1.1484,125481,2176-04-24 11:04:00,2176-04-25 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,2176-04-24 12:07:00,DIABETIC KETOACIDOSIS,0,1,F,2146-01-17 00:00:00,NaN,NaN,NaN,0
51917,77847,1.2539,139188,2164-03-25 08:37:00,2164-03-27 14:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,...,NaN,CHEST PAIN/CATH,0,1,M,2126-03-26 00:00:00,NaN,NaN,NaN,0
26265,21521,1.2861,196977,2190-08-28 06:18:00,2190-08-29 13:11:00,2190-08-29 13:11:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,2190-08-28 06:23:00,COLITIS-URINARY TRACT INFECTION,1,1,M,2108-06-29 00:00:00,2190-08-29 00:00:00,2190-08-29 00:00:00,2190-08-29 00:00:00,1
35725,29330,2.3217,152522,2144-06-24 13:37:00,2144-06-26 19:00:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,...,NaN,RESPIRATORY ARREST;S/P ERCP,0,1,M,2071-07-08 00:00:00,NaN,NaN,NaN,0
53979,84350,3.7127,172415,2199-11-18 16:44:00,2199-11-29 15:30:00,NaN,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,SUBDURAL BLEEDING,0,1,M,2146-11-23 00:00:00,NaN,NaN,NaN,0


In [13]:
dataAdm.sample(n=20)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
29190,29152,23849,102435,2188-06-10 11:45:00,2188-06-17 13:20:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT/SDA,0,1
12348,12325,10075,153175,2184-10-04 07:15:00,2184-10-11 14:50:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,PROTESTANT QUAKER,MARRIED,WHITE,NaN,NaN,AORTIC STENOSIS\BENTAL PROCEDURE /SDA,0,1
13957,12244,9994,138309,2167-08-03 07:15:00,2167-08-08 18:02:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,UNOBTAINABLE,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
20561,19000,15560,192863,2112-12-12 16:51:00,2112-12-21 11:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,JEWISH,MARRIED,WHITE,2112-12-12 11:52:00,2112-12-12 17:44:00,RULE-OUT MYOCARDIAL INFARCTION;TELEMETRY,0,1
54375,53459,82494,146806,2109-10-17 02:40:00,2109-11-01 16:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,REHAB/DISTINCT PART HOSP,Medicare,ENGL,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,2109-10-16 18:35:00,2109-10-17 03:27:00,POLYTRAUMA,0,1
44753,44622,55027,199296,2115-02-25 14:00:00,2115-03-02 18:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,NaN,NaN,AORTIC STENOSIS\? CORONARY ARTERY BYPASS GRAFT...,0,1
39617,38157,31369,170740,2176-08-25 15:04:00,2176-09-07 14:05:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,2176-08-25 12:47:00,2176-08-25 16:33:00,UPPER GI BLEED,0,1
36019,35220,28863,140125,2131-11-03 17:58:00,2131-11-04 12:50:00,2131-11-04 12:50:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,UNABLE TO OBTAIN,NaN,NaN,MYOCARDIAL INFARCTION,1,1
47816,46848,61855,189040,2104-02-28 17:17:00,2104-03-05 21:40:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2104-02-28 14:52:00,2104-02-28 18:15:00,SUBARACHNOID HEMORRHAGE,0,1
15911,15871,12947,193885,2190-08-27 11:54:00,2190-08-30 19:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,NaN,NaN,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,"DYSPNEA, CAD\CATH",0,1


In [14]:
dataPat.sample(n=20)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
28312,31705,43561,F,2079-04-20 00:00:00,NaN,NaN,NaN,0
1667,1723,1823,F,2100-12-13 00:00:00,NaN,NaN,NaN,0
29323,22142,23453,F,2184-07-20 00:00:00,NaN,NaN,NaN,0
10865,11655,12312,M,2094-03-21 00:00:00,2158-01-28 00:00:00,2158-01-28 00:00:00,2158-01-28 00:00:00,1
34324,27273,28955,F,2122-10-13 00:00:00,NaN,NaN,NaN,0
34100,36464,61355,F,2130-12-06 00:00:00,NaN,NaN,NaN,0
25077,26139,27713,M,2114-11-12 00:00:00,NaN,NaN,NaN,0
20625,15425,16314,M,2137-09-02 00:00:00,NaN,NaN,NaN,0
30168,32782,47618,F,2080-04-15 00:00:00,NaN,NaN,NaN,0
34348,27297,28981,F,2106-11-11 00:00:00,NaN,NaN,NaN,0


In [15]:
## Analyze Call out data

In [16]:
#if os.name == "posix":
dataCallOUT = pd.read_csv("C:\mimic3\CALLOUT.csv")
dataCallOUT = shuffle(dataCallOUT)
dataCallOUT.columns = [x.lower() for x in dataCallOUT.columns]
print("========CallOUT=========")
dataCallOUT.info()
print("=========================")
print()
print("========Sample=========")
dataCallOUT.sample(n=20)

========CallOUT=========
<class 'pandas.core.frame.DataFrame'>
Int64Index: 34499 entries, 17011 to 25090
Data columns (total 24 columns):
row_id                    34499 non-null int64
subject_id                34499 non-null int64
hadm_id                   34499 non-null int64
submit_wardid             34498 non-null float64
submit_careunit           5122 non-null object
curr_wardid               34498 non-null float64
curr_careunit             34498 non-null object
callout_wardid            34499 non-null int64
callout_service           34499 non-null object
request_tele              34499 non-null int64
request_resp              34499 non-null int64
request_cdiff             34499 non-null int64
request_mrsa              34499 non-null int64
request_vre               34499 non-null int64
callout_status            34499 non-null object
callout_outcome           34499 non-null object
discharge_wardid          29967 non-null float64
acknowledge_status        34499 non-null object
creat

,row_id,subject_id,hadm_id,submit_wardid,submit_careunit,curr_wardid,curr_careunit,callout_wardid,callout_service,request_tele,...,callout_status,callout_outcome,discharge_wardid,acknowledge_status,createtime,updatetime,acknowledgetime,outcometime,firstreservationtime,currentreservationtime
1300,1485,3866,162324,23.0,NaN,45.0,MICU,1,MED,0,...,Inactive,Discharged,45.0,Acknowledged,2135-01-26 10:33:55,2135-01-26 10:33:55,2135-01-26 10:39:08,2135-01-26 21:10:04,NaN,NaN
21172,22507,60947,181779,52.0,NaN,29.0,MICU,1,SURG,1,...,Inactive,Discharged,29.0,Acknowledged,2158-04-08 09:29:41,2158-04-08 13:36:02,2158-04-08 13:40:20,2158-04-08 20:40:08,2158-04-08 13:10:11,NaN
31585,30287,86276,149638,52.0,NaN,24.0,MICU,1,MED,0,...,Inactive,Discharged,24.0,Acknowledged,2174-10-12 13:51:04,2174-10-12 17:33:26,2174-10-12 17:33:28,2174-10-12 21:58:56,2174-10-12 20:29:15,NaN
16237,17627,45767,157789,52.0,NaN,48.0,MICU,48,OMED,1,...,Inactive,Discharged,48.0,Acknowledged,2110-07-22 11:44:02,2110-07-22 20:01:59,2110-07-22 21:03:19,2110-07-24 00:44:09,NaN,NaN
20455,21461,57715,178632,14.0,NaN,40.0,TSICU,40,TRAUM,1,...,Inactive,Discharged,40.0,Acknowledged,2110-02-21 10:34:58,2110-02-21 10:34:58,2110-02-21 10:50:00,2110-02-21 17:25:25,2110-02-21 11:55:27,NaN
9452,7410,19236,121214,52.0,NaN,18.0,MICU,1,MED,0,...,Inactive,Discharged,18.0,Acknowledged,2144-04-02 12:12:16,2144-04-02 12:12:16,2144-04-03 04:53:37,2144-04-03 17:10:02,NaN,NaN
12514,11721,28152,135885,52.0,NaN,18.0,MICU,1,MED,0,...,Inactive,Discharged,18.0,Acknowledged,2165-11-25 12:32:07,2165-11-25 15:03:36,2165-11-25 15:15:51,2165-11-25 20:40:41,2165-11-25 18:41:20,NaN
10513,8688,22590,172396,14.0,TSICU,14.0,TSICU,1,TRAUM,1,...,Inactive,Cancelled,NaN,Acknowledged,2178-12-27 08:59:14,2178-12-27 17:33:57,2178-12-27 12:23:47,2178-12-27 17:33:57,NaN,NaN
10669,8921,23201,131913,15.0,NaN,17.0,CSRU,17,CMED,1,...,Inactive,Discharged,17.0,Acknowledged,2181-10-06 15:19:06,2181-10-06 15:30:29,2181-10-06 15:34:47,2181-10-06 20:10:03,NaN,NaN
11667,15028,31858,188159,23.0,NaN,45.0,MICU,1,MED,0,...,Inactive,Discharged,45.0,Acknowledged,2185-10-21 11:24:58,2185-10-21 11:26:16,2185-10-21 11:55:58,2185-10-22 23:40:03,NaN,NaN


In [17]:
## concatinate tables 

In [27]:
# dataCallOUT.subject_id.duplicated().sum()
# patient id and adm id to callout is one to many 

#outcome_only = dataCallOUT[dataCallOUT['callout_outcome']==1]

#g = outcome_only.groupby(['callout_service']).size().reset_index(name='callout_service_counts')
#g['service_counts'] = data.groupby(['callout_service']).size().reset_index(name='service_counts')['service_counts']
#g['service_percent'] = g.apply(lambda row: row.callout_service_counts / row.service_counts * 100, axis=1)
#g = g.drop(columns=['callout_service_counts','service_counts'])
#g.iloc[0, g.columns.get_loc('callout_service')] = 'Female'
#g.iloc[1, g.columns.get_loc('callout_service')] = 'Male'
#g = g.set_index('gender')
#g.plot.bar(subplots=True, legend=None, ax=axes[0,0], rot=0, fontsize=12)

## Test and Training sets

## Conclusion
#### From the the following tables (ADMISSIONS, ICUSTAYS, PATIENTS) this data concludes that the following columns are the most useful in determining the length of stay of the patient. 